In [18]:
import text2emotion as te
import pandas as pd
import numpy as np

In [12]:
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")

In [ ]:
def getConversations(s,e):
    train = pd.read_csv("train.csv")
    train = train.iloc[:, :7].dropna()
    selfeval = train['selfeval']
    train["empathy"] = selfeval.str.extract('\d\|\d\|\d_(\d)\|\d\|\d')
    train = train.dropna()
    conversations = train["conv_id"].value_counts()
    conversations = conversations[s:e]
    lsconv = []
    for ids in conversations.index:
        convo = train.loc[train["conv_id"] == ids]
        convo = convo.drop(columns=["utterance_idx", "selfeval", "conv_id"])
        if len(convo["speaker_idx"].unique()) !=2:
            continue
        s1id = convo["speaker_idx"].unique()[0]
        s2id = convo["speaker_idx"].unique()[1]
        s1 = convo.loc[convo["speaker_idx"] == s1id]
        s2 = convo.loc[convo["speaker_idx"] == s2id]
        s1words = [s.replace("_comma_", ",") for s in s1["utterance"].values.tolist()]
        s2words = [s.replace("_comma_", ",") for s in s2["utterance"].values.tolist()]
        context = convo["context"].str.extract('(\w+)').values[0].item()
        escore = int(convo["empathy"].values[0])
        prompt = [convo["prompt"].unique()[0]]
        lsconv.append([context, prompt + s1words, s2words, escore])
    return lsconv

def empathy(prompt, response):
    # Gets just the emotion values for each statement.
    promptEmotions = list(te.get_emotion(prompt).values())
    responseEmotions = list(te.get_emotion(response).values())
    # Calculates the distance between the emotion vectors.
    return promptEmotions+ responseEmotions
          
def testing(s,e):
    conversations = getConversations(s,e)
    person1 = " ".join(conversations[0][1])
    person2 = " ".join(conversations[0][2])
    values = np.array(empathy(person1, person2))
    values = np.reshape(values, (1,10))
    for i in range(1, len( conversations)):
        person1 = " ".join(conversations[i][1])
        person2 = " ".join(conversations[i][2])
        temp = np.array(empathy(person1, person2))
        temp = np.reshape(temp, (1,10))
        print(temp.shape)
        print(values.shape)
        values = np.append(values, temp, axis = 0)
    return values
t = Timer()
t.start()
v = testing(0, 10)
t.stop()

c:\users\parke\onedrive\documents\github\citrusenv\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,4,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1, 10)
(1, 10)
(1, 10)
(2, 10)
(1, 10)
(3, 10)
(1, 10)
(4, 10)
(1, 10)
(5, 10)


In [25]:
v.shape

(100,)

X is a 18,108 by 10 (where the 10 is the five emotion values for speaker1 and the five emotion values for speaker2)
Y is a 18,108 by 1 (where this is just the empathy score that is in the dataset)
